In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

#from tensorflow.keras import Sequential
#from tensorflow.keras.optimizers import Adam
#from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
#from tensorflow.keras.layers import Dense, Conv2D, Flatten, GlobalAveragePooling2D, Dropout
#from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
from keras import Sequential
from keras.optimizers import Adam
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from keras.layers import Dense, Conv2D, Flatten, GlobalAveragePooling2D, Dropout
from keras.preprocessing.image import ImageDataGenerator


In [ ]:
import cv2
from PIL import Image

In [ ]:
df = pd.read_csv("E:\Data Science\Kaggle Competitions dataset\Hackerearth gala\dataset/train.csv")
df_test = pd.read_csv("E:\Data Science\Kaggle Competitions dataset\Hackerearth gala\dataset/test.csv")

In [ ]:
df.head()

In [ ]:
# True image size
IMG_WIDTH = 120
IMG_HEIGHT = 80

def preprocess_image(image, sigmaX=10):
    """
    The whole preprocessing pipeline:
    1. Read in image
    2. Apply masks
    3. Resize image to desired size
    4. Add Gaussian noise to increase Robustness
    
    :param img: A NumPy Array that will be cropped
    :param sigmaX: Value used for add GaussianBlur to the image
    
    :return: A NumPy array containing the preprocessed image
    """
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
    image = cv2.addWeighted (image,4, cv2.GaussianBlur(image, (0,0) ,sigmaX), -4, 128)
    return image

In [ ]:
y = df["Class"]

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
#img = cv2.imread(r'E:/Data Science/Kaggle Competitions dataset/Hackerearth gala/dataset/Train Images/image7042.jpg')
#cv2.imshow('image',img)
#cv2.waitKey(0)

In [ ]:
# MAKING SURE ALL IMAGES ARE OF SAME SIZE

#diff_shape = set()
PATH = "E:/Data Science/Kaggle Competitions dataset/Hackerearth gala/dataset/Train Images/"
#for i in range(df.shape[0]):
#    temp = PATH + str(df["Image"][i])
#    img = cv2.imread(temp)
#    diff_shape.add(img.shape)

In [ ]:
BATCH_SIZE = 4
# resizing the image to the size of efficientnet
IMG_WIDTH = 300
IMG_HEIGHT = 300

# Add Image augmentation to our generator
train_datagen = ImageDataGenerator(rotation_range=90,
                                   brightness_range=[0.2,1.0],
                                   zoom_range=[0.5,1.0],
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   validation_split=0.1,
                                   preprocessing_function=preprocess_image, 
                                   rescale=1 / 128.)
train_generator = train_datagen.flow_from_dataframe(df, 
                                                    x_col='Image', 
                                                    y_col='Class',
                                                    directory = PATH,
                                                    target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                    batch_size=BATCH_SIZE,
                                                    class_mode='categorical', 
                                                    subset='training')
val_generator = train_datagen.flow_from_dataframe(df, 
                                                  x_col='Image', 
                                                  y_col='Class',
                                                  directory = PATH,
                                                  target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                  batch_size=BATCH_SIZE,
                                                  class_mode='categorical',
                                                  subset='validation')

In [ ]:
## seems like efficient net is too large
# can try to freeze some layers


from keras_efficientnets import EfficientNetB3
effnet = EfficientNetB3(input_shape=(300,300,3),
                        weights='imagenet',
                        include_top=False)

In [ ]:
effnet.summary()

In [ ]:
from keras.regularizers import l2
l2 = l2(0.01)

In [ ]:
from keras.callbacks import ModelCheckpoint
mc = ModelCheckpoint(r'E:\Data Science\Kaggle Competitions dataset\Hackerearth gala\dataset/efficientnetB3_Pure.h5', monitor='categorical_crossentropy', mode='min', verbose=1, save_best_only=True)

In [ ]:
def build_model():
    """
    A custom implementation of EfficientNetB5
    for the APTOS 2019 competition
    (Regression)
    """
    model = Sequential()
    
    model.add(effnet)
    
    model.add(GlobalAveragePooling2D())
    model.add(Dense(4, activation="softmax"))
    model.compile(loss='categorical_crossentropy',
                  optimizer="adam", 
                  metrics=['categorical_crossentropy', 'mse'])
    print(model.summary())
    return model

# Initialize model
model = build_model()

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=12)
rlr = ReduceLROnPlateau(monitor='val_loss', 
                        factor=0.5, 
                        patience=5, 
                        verbose=1, 
                        mode='auto',
                        epsilon=0.00001)

# Begin training
model.fit_generator(train_generator,
                    steps_per_epoch=train_generator.samples // BATCH_SIZE,
                    epochs=90,
                    validation_data=val_generator,
                    validation_steps = val_generator.samples // BATCH_SIZE,
                    callbacks=[mc, rlr])


In [ ]:
def get_preds_and_labels(model, generator):
    """
    Get predictions and labels from the generator
    
    :param model: A Keras model object
    :param generator: A Keras ImageDataGenerator object
    
    :return: A tuple with two Numpy Arrays. One containing the predictions
    and one containing the labels
    """
    preds = []
    labels = []
    for _ in range(int(np.ceil(generator.samples / BATCH_SIZE))):
        print(_)
        x, y = next(generator)
        print(x,y)
        preds.append(model.predict(x))
        labels.append(y)
    # Flatten list of numpy arrays
    return np.concatenate(preds).ravel(), np.concatenate(labels).ravel()

In [ ]:
df_test.head(10)

In [ ]:
test_path = "E:\Data Science\Kaggle Competitions dataset\Hackerearth gala\dataset\Test Images/"

In [ ]:
df_test['Class'] = np.zeros(df_test.shape[0])
df_test["Class"] = df_test["Class"].apply(str)
df_test["Class"] = df_test["Class"].astype("category")
# For preprocessing test images
test_generator = ImageDataGenerator(preprocessing_function=preprocess_image, 
                                    rescale=1 / 128.).flow_from_dataframe(df_test,
                                                                          x_col = "Image",
                                                                          #y_col = "Class",
                                                                          directory=test_path,
                                                                          target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                                          batch_size=1,
                                                                          class_mode=None,
                                                                          shuffle=False)
filename = test_generator.filenames
number_test = len(filename)

In [ ]:
from keras.models import load_model

In [ ]:
best_model = load_model('E:\Data Science\Kaggle Competitions dataset\Hackerearth gala\dataset/efficientnetB3_no_freezing.h5')

In [ ]:
predict = best_model.predict_generator(test_generator,number_test)

In [ ]:
def convert_pred(pred):
    output = []
    output1 = []
    for i in pred:
        temp = max(i)
        for c,j in enumerate(i):
            if j == temp:
                output1.append(c)
    for i in output1:
        if i==2:
            output.append("Food")
        if i==3:
            output.append("misc")
        if i==0:
            output.append("Attire")
        if i==1:
            output.append("Decorationandsignage")
    return output1,output

In [ ]:
evaluation,submission, = convert_pred(predict)

In [ ]:
submission

In [ ]:
evaluation

In [ ]:
df_test = df_test.drop("Class",axis =1)
df_test["Class"] = submission

In [ ]:
df_test.to_csv("E:\Data Science\Kaggle Competitions dataset\Hackerearth gala\dataset/submission.csv",index = False)

In [ ]:
'''
for i in range(df_test.shape[0]):
    temp = test_path + str(df_test["Image"][i])
    print(df_test["Class"][i])
    img = cv2.imread(temp)
    cv2.imshow('image',img)
    cv2.waitKey(0)
'''